In [11]:
import awswrangler as wr
import pandas as pd
import os
import boto3
import plotly.express as px
import plotly.graph_objects as go

In [12]:
import json
with open("keys.json", "r") as f:
	aws_keys = json.load(f)

# create session
session = boto3.Session(aws_access_key_id=aws_keys["access_key"], aws_secret_access_key=aws_keys["secret_key"])
bucket_name = "labdigi-filebucket"

In [13]:
files = wr.s3.list_objects(f"s3://{bucket_name}/raw_data/")
df = pd.DataFrame(files, columns=['path'])

In [14]:
df['timestamp'] = df['path'].apply(lambda x: int(x.split("/")[-2]))

In [15]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

In [16]:
# corrigindo uma cagada
#files_to_delete = list(df[df['datetime'] < '2022-04-05']['path'])
#wr.s3.delete_objects(files_to_delete)

In [17]:
df['folder_timestamp'] = df['path'].apply(lambda x: int(x.split("/")[-3]))

In [18]:
df['folder_timestamp'].value_counts()

1649259351    697
1649425269    635
Name: folder_timestamp, dtype: int64

In [19]:
#df = df[df['folder_timestamp'] == df['folder_timestamp'].max()]
df = df[df['folder_timestamp'] == 1649259351]

In [20]:
files = df['path'].tolist()

In [60]:
list_of_dfs = []
for file in files:
	tmp_df = pd.read_json(file)
	list_of_dfs.append(tmp_df)

In [61]:
backup = list_of_dfs.copy()

In [62]:
# # open pickle
# import pickle
# with open("backup.pkl", "rb") as f:
# 	list_of_dfs = pickle.load(f)

In [63]:
df: pd.DataFrame = pd.concat(list_of_dfs)

In [64]:
df

,topic,value,time
0,grupo2-bancadaB1/hello,#5239,1649248555071
1,grupo2-bancadaB1/hello,#5240,1649248557075
2,grupo2-bancadaB1/E2,1,1649248557531
3,grupo2-bancadaB1/S4,1,1649248557853
4,grupo2-bancadaB1/E2,0,1649248558514
...,...,...,...
1,grupo2-bancadaB1/hello,#7105,1649252290161
2,grupo2-bancadaB1/hello,#7106,1649252292165
0,grupo2-bancadaB1/hello,#7107,1649252294166
1,grupo2-bancadaB1/hello,#7108,1649252296180


In [65]:
df.reset_index(drop=True, inplace=True)

In [66]:
df['time']=  df['time']*1000

In [67]:
df['datetime'] =  pd.to_datetime(df['time'], unit='us')

In [68]:
tmp_df = df[df.duplicated(subset=['topic', 'time'], keep=False)]

In [69]:
topics_times = tmp_df[['topic', 'time']].drop_duplicates()
topics_ = list(tmp_df['topic'])
times_ = list(tmp_df['time'])

# move os tempos alguns milisegundos para cima ou para baixo
for topic_, time_ in zip(topics_, times_):
	values_to_replace = []
	filtered_df = tmp_df[(tmp_df['topic'] == topic_) & (tmp_df['time'] == time_)]

	max_ = len(filtered_df)
	print("-- Topic:", topic_, "Time:", time_, "Max:", max_)
	for i in range(max_):
		index = filtered_df.iloc[i].name
		if i < max_/2:
			print((max_-i+1)*-1)
			df['time'][index] = filtered_df.iloc[i]['time'] + (max_-i+1)*-1
		if i == max_/2:
			print(0)
			df['time'][index] = filtered_df.iloc[i]['time'] + 0
		if i > max_/2:
			print((i+1)*1)
			df['time'][index] = filtered_df.iloc[i]['time'] + (i+1)*1


-- Topic: grupo2-bancadaB1/S1 Time: 1649248651140000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649248651140000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249704385000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249704385000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649249704386000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649249704386000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249704387000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249704387000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649249848208000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649249848208000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S0 Time: 1649249868106000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S0 Time: 1649249868106000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249881549000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649249881549000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649249881551000 Max: 2
-3
0
-- Topic: grupo2-bancadaB

<ipython-input-69-fd227e8446c2>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-69-fd227e8446c2>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-69-fd227e8446c2>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



 Time: 1649250110333000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649250110333000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649250116958000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S2 Time: 1649250116958000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250120905000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250120905000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250121337000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250121337000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250126544000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250126544000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250126978000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250126978000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250127444000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S1 Time: 1649250127444000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250134044000 Max: 2
-3
0
-- Topic: grupo2-bancadaB1/S3 Time: 1649250134044000 M

In [70]:
list_of_dfs = []
for topic in list(df['topic'].unique()):
	print(topic)
	tmp_df = pd.pivot(df[df['topic'] == topic], columns=['topic'], index=['time'], values=['value'])
	list_of_dfs.append(tmp_df)

grupo2-bancadaB1/hello
grupo2-bancadaB1/E2
grupo2-bancadaB1/S4
grupo2-bancadaB1/E3
grupo2-bancadaB1/S0
grupo2-bancadaB1/S5
grupo2-bancadaB1/S1
grupo2-bancadaB1/E4
grupo2-bancadaB1/S2
grupo2-bancadaB1/S7
grupo2-bancadaB1/E5
grupo2-bancadaB1/S3
grupo2-bancadaB1/E6
grupo2-bancadaB1/E1
grupo2-bancadaB1/S6


In [71]:
for i in range(len(list_of_dfs)):
	list_of_dfs[i] = list_of_dfs[i]['value']

In [72]:
# import reduce from functools
from functools import reduce
merged_df = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), list_of_dfs)

In [73]:
merged_df = merged_df.sort_index().ffill()

In [74]:
merged_df = merged_df.set_index(pd.to_datetime(merged_df.index, unit='us'), drop=True)

In [203]:
def get_cleaned_df(df, topic, time_window='250ms'):
	tmp0 = df[(df['topic'] == topic) & (df['value'] == '0')].resample(time_window).first().dropna()
	tmp1 = df[(df['topic'] == topic) & (df['value'] == '1')].resample(time_window).first().dropna()
	return pd.concat([tmp0, tmp1], axis=0).sort_index()

In [161]:
df2 = df
df = df[df['topic'] !='grupo2-bancadaB1/hello']

In [ ]:
df = df.set_index(pd.to_datetime(df['time'], unit='us'), drop=True)

# Análise dos Dados

In [346]:
def get_tmp_concat(df):
	tmp = get_cleaned_df(df, 'grupo2-bancadaB1/S4')

	if tmp.iloc[0]['value'] == '0':
		tmp = tmp.iloc[1:]

	# is even
	if len(tmp) % 2 == 0:
		pass
	# is odd
	else:
		tmp = tmp.iloc[0:len(tmp)-1]

	tmp1 = tmp[tmp['value'] == '1']['datetime']
	tmp0 = tmp[tmp['value'] == '0']['datetime']

	tmp1.reset_index(drop=True, inplace=True)
	tmp1.name = 'datetime_1'

	tmp0.reset_index(drop=True, inplace=True)
	tmp0.name = 'datetime_0'

	tmp_concat = pd.concat([tmp1, tmp0], axis=1)

	tmp_concat['timedelta'] = (tmp_concat['datetime_0']-tmp_concat['datetime_1']).dt.total_seconds()

	return tmp_concat

In [347]:
def get_modos_jogo_escolhidos(df):

	tmp_concat = get_tmp_concat(df)
	modos_jogo_escolhidos = []
	for i in range(len(tmp_concat)):
		try:
			sliced_df = df[(df['datetime'] > tmp_concat['datetime_1'].iloc[i]) & (df['datetime'] < tmp_concat['datetime_0'].iloc[i])]
			sliced_df = sliced_df[sliced_df['topic'].isin(['grupo2-bancadaB1/E3', 'grupo2-bancadaB1/E4', 'grupo2-bancadaB1/E5', 'grupo2-bancadaB1/E6'])]
			topic = sliced_df[sliced_df['value'] == '1']['topic'].iloc[0]
			datetime_ = sliced_df[sliced_df['value'] == '1']['datetime'].iloc[0]
			modos_jogo_escolhidos.append({'topic': topic, 'datetime': datetime_, 'num': i+1})
		except:
			modos_jogo_escolhidos.append({'topic': None, 'datetime': None, 'num': i+1})
	
	return modos_jogo_escolhidos

In [357]:
modos = pd.DataFrame(get_modos_jogo_escolhidos(df))

modos['label'] = modos['topic'].apply(lambda x: "Erro no jogo" if x is None else "Modo 1" if "E3" in x else "Modo 2" if "E4" in x else "Modo 3" if "E5" in x else "Modo 4")

In [358]:
modos = modos[['datetime', 'label']]

In [360]:
modos.columns = ['time', 'label']

In [368]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(modos.columns),
                fill_color='grey',
				line_color='darkslategray',
				font={'color': 'white'},
                align='center'),
    cells=dict(values=[modos.time, modos.label],
               fill_color='lightgrey',
               align='center',
			   line_color='darkslategray',
			   font={'color': 'black'}
			   )
			)
])

fig

In [363]:
statistics = pd.DataFrame(get_tmp_concat(df)['timedelta'].describe()).reset_index()
statistics.columns = ['description', 'value']

In [365]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(statistics.columns),
                fill_color='grey',
				line_color='darkslategray',
				font={'color': 'white'},
                align='center'),
    cells=dict(values=[statistics.description, statistics.value],
               fill_color='lightgrey',
               align='center',
			   line_color='darkslategray',
			   font={'color': 'black'}
			   )
			)
])

fig

## Número de Jogadas

In [270]:
# num_jogadas = df[(df['topic'] == 'grupo2-bancadaB1/S4') & (df['value'] == '1')]['time'].count()

jogadas = get_cleaned_df(df, 'grupo2-bancadaB1/S4', time_window='2s')
num_jogadas = len(jogadas[jogadas['value'] == '1'])

num_jogadas

17

## numero vezes que perdeu

In [265]:
derrotas = get_cleaned_df(df, 'grupo2-bancadaB1/S7', time_window='2s')
num_derrotas = len(derrotas[derrotas['value'] == '1'])

num_derrotas

17

In [264]:
derrotas[derrotas['value'] == '1']

,topic,value,time,datetime
time,,,,
2022-04-06 12:37:20,grupo2-bancadaB1/S7,1,1.649249e+15,2022-04-06 12:37:20.334
2022-04-06 12:38:00,grupo2-bancadaB1/S7,1,1.649249e+15,2022-04-06 12:38:01.187
2022-04-06 12:38:06,grupo2-bancadaB1/S7,1,1.649249e+15,2022-04-06 12:38:06.271
2022-04-06 12:55:10,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:55:11.507
2022-04-06 12:56:34,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:56:34.162
2022-04-06 12:56:46,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:56:46.727
2022-04-06 12:56:58,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:56:58.157
2022-04-06 12:59:16,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:59:17.458
2022-04-06 12:59:36,grupo2-bancadaB1/S7,1,1.649250e+15,2022-04-06 12:59:36.443


# numero vezes que ganhou

In [268]:
vitorias = get_cleaned_df(df, 'grupo2-bancadaB1/S6', time_window='5s')
num_vitorias = len(vitorias[vitorias['value'] == '1'])

num_vitorias

2

In [374]:
num_derrotas, num_vitorias
# create donut chart
labels = ['Vitórias', 'Derrotas']
values = [num_vitorias, num_derrotas]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title_text="Vitórias x Derrotas")
fig

# derrotas e vitórias

In [306]:
vitorias = get_cleaned_df(df, 'grupo2-bancadaB1/S6', time_window='5s')
vitorias = vitorias[vitorias['value'] == '1']
derrotas = get_cleaned_df(df, 'grupo2-bancadaB1/S7', time_window='2s')
derrotas = derrotas[derrotas['value'] == '1']
jogos = pd.concat([vitorias, derrotas], axis=0).sort_index(ascending=False)

In [307]:
jogos['label'] = jogos['topic'].apply(lambda x: "vitória" if x == 'grupo2-bancadaB1/S6'else "derrota")

In [308]:
jogos = jogos['label'].reset_index()

In [315]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(jogos.columns),
                fill_color='grey',
				line_color='darkslategray',
				font={'color': 'white'},
                align='center'),
    cells=dict(values=[jogos.time, jogos.label],
               fill_color='lightgrey',
               align='center',
			   line_color='darkslategray',
			   font={'color': 'black'}
			   )
			)
])

fig

In [376]:
fig = px.scatter(jogos, x='time', y='label', color='label', title='Vitórias x Derrotas')

# numero de rodadas

In [342]:
rodadas = get_cleaned_df(df, 'grupo2-bancadaB1/S5')
num_rodadas = len(rodadas[rodadas['value'] == '1'])

num_rodadas

hellos = df2[df2['topic'] == 'grupo2-bancadaB1/hello']
num_hellos = len(hellos)

num_hellos

1871

In [378]:
df

,topic,value,time,datetime
time,,,,
2022-04-06 12:35:57.531,grupo2-bancadaB1/E2,1,1649248557531000,2022-04-06 12:35:57.531
2022-04-06 12:35:57.853,grupo2-bancadaB1/S4,1,1649248557853000,2022-04-06 12:35:57.853
2022-04-06 12:35:58.514,grupo2-bancadaB1/E2,0,1649248558514000,2022-04-06 12:35:58.514
2022-04-06 12:37:07.682,grupo2-bancadaB1/E3,1,1649248627682000,2022-04-06 12:37:07.682
2022-04-06 12:37:08.060,grupo2-bancadaB1/S0,1,1649248628060000,2022-04-06 12:37:08.060
...,...,...,...,...
2022-04-06 13:20:39.740,grupo2-bancadaB1/S7,1,1649251239740000,2022-04-06 13:20:39.740
2022-04-06 13:24:59.917,grupo2-bancadaB1/E2,1,1649251499917000,2022-04-06 13:24:59.917
2022-04-06 13:25:00.147,grupo2-bancadaB1/E2,0,1649251500147000,2022-04-06 13:25:00.147


In [377]:
fig.write_image("vitorias_x_derrotas.png")

In [345]:
fig = go.Figure()

fig.add_trace(go.Indicator(value=num_rodadas, title={'text': 'Número de Rodadas'}, domain = {'row': 0, 'column': 0}))

fig.add_trace(go.Indicator(value=num_hellos, title={'text': 'Número de Hellos'}, domain = {'row': 1, 'column': 0}))

fig.update_layout(
    grid = {'rows': 2, 'columns': 1, 'pattern': "independent"})